In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import wradlib as wrl
from glob import glob
from numpy.lib.stride_tricks import sliding_window_view
from matplotlib import cm
from matplotlib import colors

In [5]:


dt_start = datetime.datetime(2020,1,1,0,0,0)
dt_end = datetime.datetime(2020,12,31,23,59,59)
new_index = pd.date_range(dt_start,dt_end, freq='1h')

DIR_data = 'C:/Users/sverrirhd/vedurgogn/'
URL_dates = glob(DIR_data + '??_file_dates.npy')
URL_missing_indexes = glob(DIR_data + '??_missing_indexes.npy')

dates = []
for i in range(12):
    tmp_missing = np.load(URL_missing_indexes[i])
    tmp_dates = np.load(URL_dates[i],allow_pickle=True)
    tmp_dates = np.delete(tmp_dates,tmp_missing)
    dates.append(tmp_dates)
dates = np.concatenate(dates,axis=0)

# Drainage and radar data
df_map = pd.read_pickle('../40times60_CAPPI_raw').resample('1h').sum()
df_target = pd.read_pickle('../../Drainage data/all_drainage_data_uncorrected.pkl')

# Remove/reduce effect of cycle
cycle_length = 24
remove_cycle = lambda x,c : (np.nanmax([x[c],0]) + (x.diff(c).cumsum()/c).shift(c))
remove_season = lambda x : x - x.rolling(24*7*2,min_periods = 0).min()

# for col in df_target:
#     df_target.loc[:,col] = remove_cycle(df_target.loc[:,col],cycle_length)
#     df_target.loc[:,col] = remove_season(df_target.loc[:,col])
    
target = df_target
input_2d = df_map

input_2d = input_2d.reindex(new_index)
target = target.reindex(new_index)
target = target.drop('MOS-HOL',axis='columns').interpolate(method='linear')

input_2d = input_2d.values
target = target.values

In [13]:

def in_batches(func, data, batch_size):
    n = len(data)
    n_batches = n//batch_size + int(np.ceil(n % batch_size,))
    output = []
    for i in range(n_batches):
        func_ret = func(data[i*batch_size:(i+1)*batch_size])
        output.append(func_ret)
        del func_ret
    return output

def to_rainfall(radar_data):    
    a,b = 200.,1.6
    gain,offset = 0.5, -32
    return wrl.zr.z_to_r(wrl.trafo.idecibel(offset + gain * radar_data), a=200, b=1.6)


In [15]:
# correlations = np.corrcoef(input_2d.reshape(-1,40*60),target.reshape(-1,12),rowvar=False, )
# f,axes = plt.subplots(3,4,figsize=(16,5))
# for i,ax in enumerate(axes.ravel()):
#     ax.imshow(as_rainfall_tmp[-i+1,:-12].T)
#     # ax.colorbar()
# plt.tight_layout()
# plt.show()

In [ ]:
from scipy.sparse import load_npz,save_npz

radar_csc = load_npz(DIR_data + 'radar_2020_sparse_csc_matrix.npz')

first_layer = radar_csc[:,:420*120].toarray()
second_layer = radar_csc[:,420*120:420*120*2].toarray()
third_layer = radar_csc[:,420*120*2:420*120*3].toarray()

del radar_csc

In [7]:
def rainfall_and_mean(indexes,layer):
    subset = layer[indexes.astype(int)]
    subset_rainfall = to_rainfall(subset)
    accumulated_rainfall = subset_rainfall.sum(axis=0)
    return accumulated_rainfall

def correlate_pixel(series,lag = 0):
    radar_values = np.roll(series.values,lag).reshape(-1,1)
    drainage_values = target.reshape(-1,12)
    return np.corrcoef(radar_values,drainage_values,rowvar=False)[:,0]


exclude = pd.Series(dates,dates)
index_include = pd.Series(np.arange(len(exclude)),index = exclude.index)
hourly_aggregate_indexes = index_include.groupby(pd.Grouper(freq='1h')).apply(np.array)

In [10]:
first_layer_correlation_file = 'first_layer_correlation_with_drainage_sensors.csv'

hourly_rainfall_first_layer = hourly_aggregate_indexes.apply(lambda x : rainfall_and_mean(x,first_layer))
hourly_rainfall_first_layer = pd.DataFrame(np.array((list(hourly_rainfall_first_layer.values))),hourly_rainfall_first_layer.index)
correlations = hourly_rainfall_first_layer.apply(correlate_pixel)
correlations = correlations.iloc[1:]
correlations.index = df_target.drop('MOS-HOL',axis='columns').columns
# correlations.to_csv(first_layer_correlation_file)
# correlations = pd.read_csv(first_layer_correlation_file)

NameError: name 'first_layer' is not defined

In [9]:
second_layer_correlation_file = 'second_layer_correlation_with_drainage_sensors.csv'
hourly_rainfall_second_layer = hourly_aggregate_indexes.apply(lambda x : rainfall_and_mean(x,second_layer))
hourly_rainfall_second_layer = pd.DataFrame(np.array((list(hourly_rainfall_second_layer.values))),hourly_rainfall_second_layer.index)
correlations2 = hourly_rainfall_second_layer.apply(correlate_pixel)
correlations2 = correlations2.iloc[1:]
correlations2.index = df_target.drop('MOS-HOL',axis='columns').columns
# correlations2.to_csv(second_layer_correlation_file)
# correlations2 = pd.read_csv(second_layer_correlation_file)

NameError: name 'second_layer' is not defined

In [ ]:
third_layer_correlation_file = 'third_layer_correlation_with_drainage_sensors.csv'
hourly_rainfall_third_layer = hourly_aggregate_indexes.apply(lambda x : rainfall_and_mean(x,third_layer))
hourly_rainfall_third_layer = pd.DataFrame(np.array((list(hourly_rainfall_third_layer.values))),hourly_rainfall_third_layer.index)
correlations3 = hourly_rainfall_third_layer.apply(correlate_pixel)
correlations3 = correlations3.iloc[1:]
correlations3.index = df_target.drop('MOS-HOL',axis='columns').columns
# correlations3.to_csv(third_layer_correlation_file)
# correlations3 = pd.read_csv(third_layer_correlation_file)

In [11]:

l = 5
lag_values = np.arange(-100,200,4)
for shift_0 in range(-5,5):
    for shift_1 in range(-5,5):
        means = []
        for l in lag_values:
            correlate_pixel_lag = lambda x:correlate_pixel(x,lag = l)
            # corr_vals = hourly_rainfall_first_layer.iloc[:,[5075]].apply(correlate_pixel_lag)
            corr_vals = hourly_rainfall_first_layer.iloc[:,[5075+shift_0*420 + shift_1]].apply(correlate_pixel_lag)
            means.append(corr_vals)
            
        plt.plot(lag_values,[i.iloc[5] for i in means])
        print(lag_values[np.argmax([i.iloc[5] for i in means])],end=',')


NameError: name 'hourly_rainfall_first_layer' is not defined

In [ ]:
# tmp_2d = correlations2.loc['RVK-SKE'].values.reshape((420,120))
# window_size = (50,50)
# tmp_sliding_windows = sliding_window_view(tmp_2d,window_size)
# best_window = tmp_sliding_windows.sum(axis=(-1,-2)).reshape(-1).argmax()

# tmp_area = tmp_sliding_windows.reshape((-1,) + window_size)[best_window]
# max_0 = np.max(np.abs(correlations3.loc['RVK-SKE']))
# divnorm_0 = colors.TwoSlopeNorm(vmin=-max_0, vcenter=0., vmax=max_0)
# plt.imshow(tmp_area,cmap='RdBu',norm=divnorm_0)
# tmp_sliding_windows.shape,best_window



In [ ]:
# best_pixel_BOD = correlations2.loc['RVK-BOD'].argmax()
# best_pixel_BOD_values = second_layer[:,best_pixel_BOD]

# plt.plot(dates,best_pixel_BOD_values)
# plt.plot(df_target.loc[new_index,'RVK-BOD'])
# plt.show()
# hourly_sum_BOD = pd.DataFrame(best_pixel_BOD_values,dates).resample('1h').sum().loc[new_index]
# plt.scatter(hourly_sum_BOD,df_target.loc[new_index,'RVK-BOD'],s = 0.1)
# plt.show()

In [ ]:
# plt.plot(pd.Series(best_pixel_VES_values,dates).groupby(pd.Grouper(freq='1h')).sum())
# plt.twinx().plot(df_target.loc[new_index,'RVK-VES'],color='red')
# plt.xlim(datetime.datetime(2020,1,1),datetime.datetime(2020,2,1))
# plt.show()

In [ ]:
# best_pixel_VES = correlations.loc['RVK-VES'].argmax()
# best_pixel_VES_values = first_layer[:,best_pixel_VES]

# plt.plot(dates,best_pixel_VES_values)
# plt.plot(df_target.loc[new_index,'RVK-VES'])
# plt.show()

# plt.plot(dates,best_pixel_VES_values)
# plt.plot(df_target.loc[new_index,'RVK-VES'])
# plt.xlim(datetime.datetime(2020,1,1),datetime.datetime(2020,2,1))
# plt.show()

# hourly_sum_VES = pd.DataFrame(best_pixel_VES_values,dates).resample('1h').sum().loc[new_index]
# plt.scatter(hourly_sum_VES,df_target.loc[new_index,'RVK-VES'],s = 0.1)
# plt.show()

In [ ]:
# df_inputs.temperature.plot()
# print(np.corrcoef(df_inputs.radiation.fillna(0).values,df_target.loc[new_index,'RVK-VES'].fillna(method='ffill')))
# print(np.corrcoef(df_inputs.moist_pressure.fillna(0).values,df_target.loc[new_index,'RVK-VES'].fillna(method='ffill')))
# df_target.loc[new_index,'RVK-VES'].plot()

In [ ]:
# df_inputs = pd.read_pickle('./2020_input_data_four_areas')
# df_inputs.head()
# plt.figure(figsize=(15,5))
# df_target.loc[new_index,'RVK-VES'].plot()


In [ ]:

f,axes = plt.subplots(12,3,figsize=(8*3,12*8))
r_vec = np.arange(0,240000,2000) + 1000
az_vec = np.arange(0,360+1,0.86)
for i in range(12):
    
    arr_0 = correlations.values[i,:].reshape(420,120)
    arr_1 = correlations2.values[i,:].reshape(420,120)
    arr_2 = correlations3.values[i,:].reshape(420,120)
    
    max_0 = np.max(np.abs(arr_0))
    max_1 = np.max(np.abs(arr_1))
    max_2 = np.max(np.abs(arr_2))
    
    divnorm_0 = colors.TwoSlopeNorm(vmin=-max_0, vcenter=0., vmax=max_0)
    divnorm_1 = colors.TwoSlopeNorm(vmin=-max_1, vcenter=0., vmax=max_1)
    divnorm_2 = colors.TwoSlopeNorm(vmin=-max_2, vcenter=0., vmax=max_2)
    
    img0 = wrl.vis.plot_ppi(arr_0, r = r_vec, az = az_vec, ax=axes[i,0],cmap='RdBu',norm=divnorm_0)
    img1 = wrl.vis.plot_ppi(arr_1, r = r_vec, az = az_vec, ax=axes[i,1],cmap='RdBu',norm=divnorm_1)
    img2 = wrl.vis.plot_ppi(arr_2, r = r_vec, az = az_vec, ax=axes[i,2],cmap='RdBu',norm=divnorm_2)

    # plt.imshow(correlations.values[i,:].reshape(420,120).T)
    plt.colorbar(img0[1],ax = axes[i,0])
    plt.colorbar(img1[1],ax = axes[i,1])
    plt.colorbar(img2[1],ax = axes[i,2])
    axes[i,0].set_title(df_target.columns[i] + ' - ' + 'First layer')
    axes[i,1].set_title(df_target.columns[i] + ' - ' + 'Second layer')
    axes[i,2].set_title(df_target.columns[i] + ' - ' + 'Third layer')

plt.tight_layout()
plt.savefig('./radar_drainage_correlation_map.png')
plt.close()
# plt.show()

# plot correlations over geographical locations

In [ ]:

import geopandas as gpd
from pyproj import Transformer

transformer_gps_isl = Transformer.from_crs("epsg:4326", "epsg:3057" ) # Umbreyta GPS staðsetningu yfir í cartesian hnit fyrir Ísland
transformer_isl_gps = Transformer.from_crs("epsg:3057", "epsg:4326") # Umbreyta cartesian hnitum aftur í GPS hnit

DIR_map = 'C:/Users/sverrirhd/Google Drive/Skóli/DTU/Thesis/Programming/Example code/Jonas example of rain data'
DIR_data = 'C:/Users/sverrirhd/vedurgogn/'

bg_map_file = f'{DIR_map}/Iceland_map/geoBoundariesSimplified-3_0_0-ISL-ADM0.shp'
URL_pixel_coords_save = DIR_data + 'radar_pixel_coords.npy'
URL_radar_coords_save = DIR_data + 'radar_raw_pixel_coords_gps.npy'
radar_coords = np.load(URL_pixel_coords_save)
radar_raw_gps_coords = np.load(URL_radar_coords_save)


plot_title = 'Correlations over Reykjavík'

bg_map = gpd.read_file(bg_map_file)

## Recreate the CAPPI patches and extract their locations

In [ ]:
ix,iy = 188168//602,188168%602
x_shift = 1
y_shift = -1
print('pixel center:',ix,iy)
X_margin = 20 #  pixels
y_margin = 30 #  pixels
H,W = 2*X_margin,2*y_margin
ix_start = ix + x_shift - X_margin
ix_end = ix + x_shift + X_margin
iy_start = iy + y_shift - y_margin
iy_end = iy + y_shift + y_margin

radar_patch_coords = radar_coords.reshape((602,602,-1))[ix_start:ix_end,iy_start:iy_end,:2].reshape(-1,2)
radar_patch_coords_gps = [transformer_isl_gps.transform(a,b) for a,b in radar_patch_coords]
radar_patch_coords_gps = np.array(radar_patch_coords_gps)

In [ ]:

radar_patches = []
for i in range(12):
    radar_tmp = np.load(DIR_data + f'CAPPI_{i}.npy')
    radar_tmp = radar_tmp.reshape(-1,602,602)
    tmp_patches = radar_tmp[:,ix_start:ix_end,iy_start:iy_end]
    tmp_rainfall_patch = to_rainfall(tmp_patches)
    radar_patches.append(tmp_rainfall_patch)
radar_patches = np.concatenate(radar_patches,axis = 0)


In [ ]:
def correlate_pixel(series,lag = 0):
    radar_values = np.roll(series.values,lag).reshape(-1,1)
    drainage_values = target.reshape(-1,12)
    return np.corrcoef(radar_values,drainage_values,rowvar=False)[:,0]

l = 1
correlate_pixel_lag = lambda x:correlate_pixel(x,lag = l)
correlation_grid = df_map.apply(correlate_pixel_lag).iloc[1:]
correlation_grid.index = df_target.drop('MOS-HOL',axis='columns').columns

In [ ]:
f,axes = plt.subplots(3,4,figsize=(16,12))

for i,ax in enumerate(axes.ravel()):
    im = ax.imshow(correlation_grid.iloc[i].values.reshape((H,W))[::-1])
    plt.colorbar(im, ax=ax)

In [ ]:
x_coords = radar_patch_coords_gps[:,1].reshape(H,W)
y_coords = radar_patch_coords_gps[:,0].reshape(H,W)

In [ ]:
lag_values = range(-10,10)
for l in lag_values:
    correlate_pixel_lag = lambda x:correlate_pixel(x,lag = l)
    correlation_grid = df_map.apply(correlate_pixel_lag).iloc[1:]
    correlation_grid.index = df_target.drop('MOS-HOL',axis='columns').columns

    f,axes = plt.subplots(4,3,figsize=(3*10,4*7))

    for i,ax in enumerate(axes.ravel()):
        arr = correlation_grid.iloc[i].values.reshape(H,W)
        # arr_max = np.max(np.abs(arr))
        divnorm = colors.TwoSlopeNorm(vcenter=0.,)
        bg_map.plot(edgecolor="black",facecolor='lightgrey',ax = ax)
        img = ax.pcolormesh(x_coords,
                            y_coords,
                            arr[:-1,:-1],
                            shading='flat',
                            alpha = 0.65,
                            cmap='RdBu',
                            norm=divnorm
                            ) #
                
        
        ax.set_xlim(-22.6,-21.25)
        ax.set_ylim(63.95,64.35)
        ax.set_title(correlation_grid.index[i],fontsize=20)
        plt.colorbar(img,ax = ax)
        
    plt.tight_layout()
    plt.savefig(f'./cappi_correlation_lag_map_series/Correlations_overlayed_on_map_lag{l}.png')
    plt.close()

# And now with the raw radar pixels instead

In [ ]:
# plt.imshow(hourly_rainfall_second_layer.iloc[0,:].values.reshape(420,120)[])

In [ ]:
# indexes_of_interest = np.arange(hourly_rainfall_first_layer.shape[-1]).reshape(420,120)[:,:40].ravel()
# correlate_pixel_lag = lambda x:correlate_pixel(x,lag = 44)
# correlation_grid = hourly_rainfall_second_layer.iloc[:,indexes_of_interest].apply(correlate_pixel_lag).iloc[1:]
# correlation_grid.index = df_target.drop('MOS-HOL',axis='columns').columns

In [ ]:
# def compute_cell_edges(x_coords,y_coords):
#     x_diff = x_coords[:-1] - x_coords[1:]
#     x_diff = np.append(x_diff,x_diff[-1])
#     x_corner = x_coords - x_diff/2
#     x_corner = np.append(x_corner, x_corner[-1 + ])
    
#     y_diff = y_coords[:-1] - y_coords[1:]
#     y_diff = np.append(y_diff,y_diff[-1])
#     y_corner = y_coords - y_diff/2
    
    
    
#     return x_coords - cell_start
    

In [ ]:
r_vec = np.arange(0,240000,2000) + 1000
az_vec = np.arange(0,360+1,0.86)
range_from = 10
range_to = 35
angle_from = int((420/42)*5)
angle_to = int(420/4 + 420//36) 

indexes_of_interest = np.arange(hourly_rainfall_first_layer.shape[-1]).reshape(420,120)[angle_from:angle_to,range_from:range_to].ravel()

coords_of_interest_0 = radar_raw_gps_coords[indexes_of_interest]
coords_of_interest_1 = radar_raw_gps_coords[indexes_of_interest + 50400]
coords_of_interest_2 = radar_raw_gps_coords[indexes_of_interest + 2*50400]

x_coords_radar_0 = coords_of_interest_0[:,1].reshape(angle_to-angle_from,range_to-range_from)
y_coords_radar_0 = coords_of_interest_0[:,0].reshape(angle_to-angle_from,range_to-range_from)

x_coords_radar_1 = coords_of_interest_1[:,1].reshape(angle_to-angle_from,range_to-range_from)
y_coords_radar_1 = coords_of_interest_1[:,0].reshape(angle_to-angle_from,range_to-range_from)

x_coords_radar_2 = coords_of_interest_2[:,1].reshape(angle_to-angle_from,range_to-range_from)
y_coords_radar_2 = coords_of_interest_2[:,0].reshape(angle_to-angle_from,range_to-range_from)



In [ ]:
# ax = plt.gca()
# bg_map.plot(edgecolor="black",facecolor='lightgrey',ax=ax)
# img0 = ax.pcolormesh(x_coords_radar_0,
#                       y_coords_radar_0,
#                       correlation_grid_0,
#                             shading='nearest',
#                             alpha = 0.65,
#                             cmap='RdBu',
#                             norm=divnorm) #

# ax.set_xlim(-22.6,-21.25)
# ax.set_ylim(63.95,64.35)
# ax.set_title(correlation_grid.index[i],fontsize=20)

In [ ]:
# hourly_rainfall_first_layer

In [ ]:
corr_dict = {}

corr_grid_index = df_target.drop('MOS-HOL',axis='columns').columns
lag_values = np.arange(-10,10,1)
# for l in np.arange(0,45,5):
for l in lag_values:
    print(l)
    if (l,2) in corr_dict.keys():
        continue
    else: 
        correlate_pixel_lag = lambda x:correlate_pixel(x,lag = l)
        correlation_grid_0 = hourly_rainfall_first_layer.iloc[:,indexes_of_interest].apply(correlate_pixel_lag).iloc[1:]
        correlation_grid_1 = hourly_rainfall_second_layer.iloc[:,indexes_of_interest].apply(correlate_pixel_lag).iloc[1:]
        correlation_grid_2 = hourly_rainfall_third_layer.iloc[:,indexes_of_interest].apply(correlate_pixel_lag).iloc[1:]
        
        corr_dict[l,0] = correlation_grid_0
        corr_dict[l,1] = correlation_grid_1
        corr_dict[l,2] = correlation_grid_2
        
    
    # for i in range(12):
    #     print(l,i)
    #     if (l,i,0) in corr_dict.keys():
    #         continue
    #     correlation_grid_0_curr = correlation_grid_0.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)
    #     correlation_grid_1_curr = correlation_grid_1.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)
    #     correlation_grid_2_curr = correlation_grid_2.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)

    #     corr_dict[(l,i,0)] = correlation_grid_0_curr
    #     corr_dict[(l,i,1)] = correlation_grid_1_curr
    #     corr_dict[(l,i,2)] = correlation_grid_2_curr




In [ ]:
RVK_VES = pd.DataFrame([corr_dict[i].iloc[7] for i in corr_dict if i[1] == 1], index = lag_values)
RVK_VES_tensor = RVK_VES.values.reshape((len(lag_values),angle_to-angle_from,range_to-range_from))

In [ ]:
for i in RVK_VES_tensor:
    plt.imshow(i.T)
    plt.show()

In [ ]:
map_coords_tmp = np.transpose(np.array([x_coords_radar_1,y_coords_radar_1,]),(1,2,0))
coords_list = []
for i in lag_values:
    lag_coord = np.full(map_coords_tmp.shape[:-1] + (1,),i)*100
    lag_layer_list = np.concatenate([map_coords_tmp,lag_coord],-1)
    coords_list.append(lag_layer_list)
    
tensor_coords = np.array(coords_list)

In [ ]:
cmap = cm.ScalarMappable(norm = colors.Normalize(vmin=-0.04, vmax=0.2))
# cmap = cm.ScalarMappable(cmap = cm.get_cmap('seismic'))
# cmap = cm.get_cmap('seismic')

In [ ]:
norm = colors.Normalize(vmin=-0.04, vmax=0.2)
cmap = cm.seismic

m = cm.ScalarMappable(norm=norm, cmap=cmap)


In [ ]:
# import geopandas as gpd
# import shapely

# iceland_polygons = []
# iceland_polygons_isl = []

# for poly in iceland_polygons_tmp:
#     poly_gps = np.array(poly.exterior.coords.xy).T[:,[1,0]]
#     poly_isl = [transformer_gps_isl.transform(x,y) for x,y in poly_gps]
#     iceland_polygons.append(shapely.geometry.polygon.Polygon(poly_gps))
#     iceland_polygons_isl.append(shapely.geometry.polygon.Polygon(poly_isl))
# mainland_id = np.argmax([x.area for x in iceland_polygons])
    
# points = np.array(iceland_polygons_isl[mainland_id].exterior.coords.xy).T
# points = np.c_[points,np.zeros(len(points)) + 0.12]
# lines = np.c_[np.arange(0,len(points)),np.arange(1,len(points)+1)]
# lines[-1,1] = 0

# # colors = [[1, 0, 0] for i in range(len(lines))]
# line_set = o3d.geometry.LineSet(
#     points=o3d.utility.Vector3dVector(points),
#     lines=o3d.utility.Vector2iVector(lines),
# )

In [ ]:
# import open3d as o3d
# points_ = tensor_coords.reshape(-1,3)
# points_ = np.array([transformer_gps_isl.transform(b,a) + (c,) for a,b,c in points_])
# colors_ = m.to_rgba(RVK_VES_tensor.reshape(-1))[:,:-1]

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points_)
# pcd.colors = o3d.utility.Vector3dVector(colors_)

# o3d.visualization.draw_geometries( [line_set,pcd])

# Blob detection in 3D


In [ ]:
print([(i,a) for i,a in enumerate(df_target.columns)])

In [ ]:
params = []
for sensor_index in range(12):
    for layer in range(3):
        for agg_func in [np.sum,np.max,np.min,np.argmax,np.argmin]:
            params.append((sensor_index,layer,agg_func))
# np.random.shuffle(params)

In [ ]:
from sklearn.mixture import GaussianMixture


# sensor_index = 7
# layer = 2
# agg_func = np.sum
###

for sensor_index,layer,agg_func in params:
    sensor_name = df_target.drop('MOS-HOL',axis='columns').columns[sensor_index]
    file_name = sensor_name + '_' + agg_func.__name__ + '_' + f'layer{layer}'
    
    # if glob(file_name)
    search_string = './*/'+file_name+'*'
    if len(glob(search_string)) > 0:
        continue
    if layer == 0:
        x_coords_tmp,y_coords_tmp = x_coords_radar_0,y_coords_radar_0
    if layer == 1:
        x_coords_tmp,y_coords_tmp = x_coords_radar_1,y_coords_radar_1
    if layer == 2:
        x_coords_tmp,y_coords_tmp = x_coords_radar_2,y_coords_radar_2


    RVK_sensor = pd.DataFrame([corr_dict[i].iloc[sensor_index] for i in corr_dict if i[1] == layer], index = lag_values)
    RVK_sensor_tensor = RVK_sensor.values.reshape((len(lag_values),angle_to-angle_from,range_to-range_from))


    plt.figure(figsize=(10,6))
    ax=plt.gca()
    bg_map.plot(edgecolor="black",facecolor='lightgrey',ax=ax)
    img = ax.pcolormesh(x_coords_tmp,
                        y_coords_tmp,
                        agg_func(RVK_sensor_tensor,axis=0),
                            shading='flat',
                            alpha = 0.65,
                            # norm=divnorm
                            ) #
    plt.colorbar(img,ax = ax)
    ax.set_title(file_name)
    ax.set_xlim(-22.6,-21.25)
    ax.set_ylim(63.95,64.35)

    
    plt.savefig(f'./correlation_agg/{file_name}.png')
    # plt.show()
    plt.close()

In [ ]:
RVK_VES_tensor.shape

In [ ]:

gm = GaussianMixture(n_components=4, n_init = 10, max_iter = 100).fit(RVK_VES_tensor.reshape(-1,1))
classifications = gm.predict(RVK_VES_tensor.reshape(-1,1))
classification_tensor = classifications.reshape(RVK_VES_tensor.shape)

for class_ in range(4):
    bg_map.plot(edgecolor="black",facecolor='lightgrey')
    plt.pcolormesh(x_coords_radar_0,
                y_coords_radar_0,
                (classification_tensor == class_).sum(axis=0),
                            shading='nearest',
                            alpha = 0.65,
                            # norm=divnorm
                            ) #

    plt.title(df_target.columns[sensor_index])
    plt.xlim(-22.6,-21.25)
    plt.ylim(63.95,64.35)
    plt.show()

In [ ]:
plt.imshow(classification_tensor.mean(axis=0))

In [ ]:
plt.hist(RVK_VES_tensor.ravel(),bins=np.linspace(-0.05,0.2,100))
plt.vlines(gm.means_,0,3000,color='red')
plt.show()

In [ ]:
corr_dict.keys()

In [73]:

iterator = 0
for l in lag_values:  
    
    iterator +=1
    f,axes = plt.subplots(12,3,figsize=(3*8,12*5))
        
    correlation_grid_0 = corr_dict[(l,0)]
    correlation_grid_1 = corr_dict[(l,1)]
    correlation_grid_2 = corr_dict[(l,2)]
    
    if (l,0) in corr_dict.keys():
        for i in range(12):
            correlation_grid_0_curr = correlation_grid_0.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)
            correlation_grid_1_curr = correlation_grid_1.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)
            correlation_grid_2_curr = correlation_grid_2.iloc[i].values.reshape(angle_to-angle_from,range_to-range_from)

            
            divnorm = colors.TwoSlopeNorm(vcenter=0.,)
            bg_map.plot(edgecolor="black",facecolor='lightgrey',ax = axes[i,0])
            bg_map.plot(edgecolor="black",facecolor='lightgrey',ax = axes[i,1])
            bg_map.plot(edgecolor="black",facecolor='lightgrey',ax = axes[i,2])
            
            img0 = axes[i,0].pcolormesh(x_coords_radar_0,
                                    y_coords_radar_0,
                                    correlation_grid_0_curr,
                                    shading='flat',
                                    alpha = 0.65,
                                    cmap='seismic',
                                    norm=divnorm
                                    ) #
            
            img1 = axes[i,1].pcolormesh(x_coords_radar_1,
                                        y_coords_radar_1,
                                        correlation_grid_1_curr,
                                        shading='flat',
                                        alpha = 0.65,
                                        cmap='seismic',
                                        norm=divnorm
                                        ) #
                
            img2 = axes[i,2].pcolormesh(x_coords_radar_2,
                                        y_coords_radar_2,
                                        correlation_grid_2_curr,
                                        shading='flat',
                                        alpha = 0.65,
                                        cmap='seismic',
                                        norm=divnorm
                                        ) #
            
            axes[i,0].set_xlim(-22.6,-21.25)
            axes[i,0].set_ylim(63.95,64.35)
            axes[i,0].set_title(correlation_grid.index[i],fontsize=20)
            
            axes[i,1].set_xlim(-22.6,-21.25)
            axes[i,1].set_ylim(63.95,64.35)
            axes[i,1].set_title(correlation_grid.index[i],fontsize=20)
            
            axes[i,2].set_xlim(-22.6,-21.25)
            axes[i,2].set_ylim(63.95,64.35)
            axes[i,2].set_title(correlation_grid.index[i],fontsize=20)
            
            plt.colorbar(img0, ax = axes[i,0])
            plt.colorbar(img1, ax = axes[i,1])
            plt.colorbar(img2, ax = axes[i,2])
        
    plt.tight_layout()
    plt.savefig(f'./pixel_correlation_lag_map_series/Correlations_overlayed_on_map_{iterator}_lag_{l}.png')
    plt.close()

KeyboardInterrupt: 

In [ ]:
# ax = plt.gca()
# bg_map.plot(edgecolor="black",facecolor='lightgrey',ax = ax)
# img = ax.pcolormesh(x_coords_radar,
#                     y_coords_radar,
#                     correlation_grid_0,
#                     shading='nearest',
#                     alpha = 0.65,
#                     cmap='RdBu',
#                     norm=divnorm) #
        
# ax.set_xlim(-22.6,-21.25)
# ax.set_ylim(63.95,64.35)
# plt.show()


# f,axes = plt.subplots(12,3,figsize=(8*3,12*8))
# r_vec = np.arange(0,240000,2000) + 1000
# az_vec = np.arange(0,360+1,0.86)
# for i in range(12):
    
#     arr_0 = correlations.values[i,:].reshape(420,120)
#     arr_1 = correlations2.values[i,:].reshape(420,120)
#     arr_2 = correlations3.values[i,:].reshape(420,120)
    
#     max_0 = np.max(np.abs(arr_0))
#     max_1 = np.max(np.abs(arr_1))
#     max_2 = np.max(np.abs(arr_2))
    
#     divnorm_0 = colors.TwoSlopeNorm(vmin=-max_0, vcenter=0., vmax=max_0)
#     divnorm_1 = colors.TwoSlopeNorm(vmin=-max_1, vcenter=0., vmax=max_1)
#     divnorm_2 = colors.TwoSlopeNorm(vmin=-max_2, vcenter=0., vmax=max_2)
    
#     img0 = wrl.vis.plot_ppi(arr_0, r = r_vec, az = az_vec, ax=axes[i,0],cmap='RdBu',norm=divnorm_0)
#     img1 = wrl.vis.plot_ppi(arr_1, r = r_vec, az = az_vec, ax=axes[i,1],cmap='RdBu',norm=divnorm_1)
#     img2 = wrl.vis.plot_ppi(arr_2, r = r_vec, az = az_vec, ax=axes[i,2],cmap='RdBu',norm=divnorm_2)

#     # plt.imshow(correlations.values[i,:].reshape(420,120).T)
#     plt.colorbar(img0[1],ax = axes[i,0])
#     plt.colorbar(img1[1],ax = axes[i,1])
#     plt.colorbar(img2[1],ax = axes[i,2])
#     axes[i,0].set_title(df_target.columns[i] + ' - ' + 'First layer')
#     axes[i,1].set_title(df_target.columns[i] + ' - ' + 'Second layer')
#     axes[i,2].set_title(df_target.columns[i] + ' - ' + 'Third layer')

# plt.tight_layout()
# plt.savefig('./radar_drainage_correlation_map.png')
# plt.close()
# # plt.show()